# 台股選股清單  
## 選股條件  
* KD值低於30，五日平均交易量大於1000張，本益比小於20倍

In [1]:
import os
import sys
from datetime import date
from datetime import datetime
from pathlib import Path

import pandas as pd

In [2]:
from finlab import data
import finlab

In [3]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.google_api import gspread_manager as gsm
from proj_util_pkg.common import tw_stock_topic as tst

✅ 成功載入環境變數檔案: /Users/bryson0083/projects/TuaThanTsinn/src/TuaThanTsinn/proj_util_pkg/config/.env
[環境變數]PROJECT_ROOT: /Users/bryson0083/projects/TuaThanTsinn/src/TuaThanTsinn


## 公用參數設定

In [4]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

輸入成功!


In [5]:
# 資訊輸出Google SpreadSheet 表單參數設定
GSPERAD_SHEET_KEY = os.environ.get('gspread_wb_key')  # Google SpreadSheet 表單ID
OUTPUT_GSHEET_NAME = '選股清單01'

In [6]:
# 本地報表輸出路徑
REPORT_PATH = os.environ.get('report_path')

## 外部資料讀取

In [7]:
# 讀取台股收盤價資訊
close = data.get("price:收盤價", save_to_storage=True)
vol = data.get("price:成交股數", save_to_storage=True)
stock_info = data.get('company_basic_info', save_to_storage=True)
pe_ratio = data.get('price_earning_ratio:本益比', save_to_storage=True)
pb_ratio = data.get('price_earning_ratio:股價淨值比', save_to_storage=True)
institutional_investors_foreign = data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)', save_to_storage=True)
institutional_investors_inv_trust = data.get('institutional_investors_trading_summary:投信買賣超股數', save_to_storage=True)
institutional_investors_dealer = data.get('institutional_investors_trading_summary:自營商買賣超股數(避險)', save_to_storage=True)

Your version is 1.5.6, please install a newer version.
Use "pip install finlab==1.5.7" to update the latest version.


## 數據分析

In [8]:
start_time = datetime.now()
print(f"ANA001_台股選股 「KD選股」 分析開始時間: {start_time}")

ANA001_台股選股 「KD選股」 分析開始時間: 2026-02-27 16:24:21.964181


In [9]:
stock_name = stock_info[['stock_id', '公司簡稱']]
stock_name = stock_name.rename(columns={'stock_id': 'symbol'})

In [10]:
# stock_name

In [11]:
sma18 = close.average(18)
sma50 = close.average(50)
vol_avg5 = vol.average(5)

In [12]:
# # 篩選連三漲
# rise18_df = sma18.rise().sustain(3).tail(1)

In [13]:
# rise18_df

In [14]:
# 條件篩選
df2 = sma18 > sma50

today = date.today().strftime("%Y-%m-%d")
# today = "2024-05-31"
# filtered_df2 = df2[df2.index == today]
filtered_df2 = df2.tail(1)

filtered_df2

symbol,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2026-02-26,False,True,True,True,True,False,True,True,True,False,...,False,False,False,True,True,False,True,False,True,True


In [15]:
filtered_symbols = filtered_df2.columns[filtered_df2.iloc[0].values].tolist()
filtered_symbols

['0050',
 '0051',
 '0052',
 '0053',
 '0055',
 '0056',
 '0057',
 '0061',
 '006201',
 '006203',
 '006204',
 '006205',
 '006206',
 '006207',
 '006208',
 '00631L',
 '00634R',
 '00635U',
 '00638R',
 '00639',
 '00640L',
 '00642U',
 '00643',
 '00645',
 '00646',
 '00647L',
 '00650L',
 '00654R',
 '00656R',
 '00657',
 '00657K',
 '00660',
 '00661',
 '00663L',
 '00665L',
 '00668',
 '00668K',
 '00671R',
 '00675L',
 '00678',
 '00679B',
 '00680L',
 '00684R',
 '00685L',
 '00687B',
 '00687C',
 '00688L',
 '00690',
 '00692',
 '00693U',
 '00700',
 '00701',
 '00702',
 '00706L',
 '00708L',
 '00709',
 '00711B',
 '00712',
 '00713',
 '00714',
 '00715L',
 '00717',
 '00728',
 '00731',
 '00733',
 '00735',
 '00736',
 '00738U',
 '00739',
 '00740B',
 '00751B',
 '00754B',
 '00755B',
 '00756B',
 '00758B',
 '00759B',
 '00761B',
 '00762',
 '00763U',
 '00764B',
 '00768B',
 '00771',
 '00772B',
 '00775B',
 '00779B',
 '00780B',
 '00783',
 '00786B',
 '00787B',
 '00792B',
 '00793B',
 '00795B',
 '00799B',
 '00836B',
 '00840B',

In [16]:
slowk, slowd = data.indicator(
    "STOCH",
    adjust_price=False,
    resample="D",
    fastk_period=9,
    slowk_period=3,
    slowk_matype=0,
    slowd_period=3,
    slowd_matype=0,
    save_to_storage=True,
)

In [17]:
# 選股策略： 最近一個交易日，ＫＤ值低於３０且五日平均成交量大於１０００張，本益比小於20
kd_low_df = (slowd < 30) & (slowk < 30) & (vol_avg5 >= 1000000) & (pe_ratio <= 20)
kd_low_df = kd_low_df.tail(1)

filtered_symbols = kd_low_df.columns[kd_low_df.iloc[0]].tolist()
df_filtered_symbols = pd.DataFrame(filtered_symbols, columns=['symbol'])

df_filtered_symbols

,symbol
0,1229
1,1319
2,1795
3,2474
4,2520
5,2608
6,2610
7,2618
8,4167
9,4763


In [18]:
# Assuming you have a DataFrame called stock_name with columns stock_id and stock_name

# Merge df_filtered_symbols with stock_name on stock_id
merged_df = df_filtered_symbols.merge(stock_name, on='symbol')

# Print the merged DataFrame
print(merged_df)

   symbol    公司簡稱
0    1229      聯華
1    1319      東陽
2    1795      美時
3    2474      可成
4    2520      冠德
5    2608    嘉里大榮
6    2610      華航
7    2618     長榮航
8    4167     松瑞藥
9    4763  材料*-KY
10   5508     永信建
11   5534      長虹
12   6179      亞通
13   6189      豐藝
14   6245      立端
15   6472      保瑞
16   6757    台灣虎航
17   6890   來億-KY
18   6962   奕力-KY
19   9941      裕融


In [19]:
# 取得篩選股票的本益比
# Step 1: Extract symbols from merged_df
symbols = merged_df['symbol'].tolist()

# Step 2: Filter pe_ratio DataFrame using symbols
filtered_pe_ratio = pe_ratio[symbols]

# Step 3: Retrieve the latest value for each symbol
latest_pe_ratio = filtered_pe_ratio.iloc[-1]

# Print the latest pe_ratio values
print(latest_pe_ratio)


symbol
1229    17.52
1319    15.28
1795    15.47
2474    14.01
2520     8.96
2608    15.02
2610     8.50
2618     7.88
4167    11.84
4763     6.18
5508    14.88
5534    10.24
6179    15.43
6189    16.96
6245    13.13
6472    19.49
6757    12.98
6890    14.50
6962    13.01
9941    11.40
Name: 2026-02-26 00:00:00, dtype: float64


In [20]:
df = pd.DataFrame(latest_pe_ratio).reset_index()
df.columns = ['symbol', 'pe_ratio']

# Merge df_filtered_symbols with stock_name on stock_id
merged_df = merged_df.merge(df, on='symbol')
merged_df.rename(
    columns={
        'symbol': '股票代號', 
        'pe_ratio': '本益比'
    },
    inplace=True
)
merged_df["web_link"] = merged_df["股票代號"].apply(lambda x: f"https://www.wantgoo.com/stock/{x}/technical-chart")
merged_df["題材概念股"] = merged_df["股票代號"].apply(lambda x: tst.read_topic_stocks(x))

merged_df

,股票代號,公司簡稱,本益比,web_link,題材概念股
0,1229,聯華,17.52,https://www.wantgoo.com/stock/1229/technical-c...,"'台積電概念股', '氫能概念股', '都市更新概念股', '智慧城市展概念股', '傳產內..."
1,1319,東陽,15.28,https://www.wantgoo.com/stock/1319/technical-c...,"'大陸收成股', '台幣貶值受惠概念股', '特斯拉概念股', '臺灣中型100指數成分股'..."
2,1795,美時,15.47,https://www.wantgoo.com/stock/1795/technical-c...,"'台商美國概念股', '再生醫療概念股', '情人節概念股', '臺灣中型100指數成分股'..."
3,2474,可成,14.01,https://www.wantgoo.com/stock/2474/technical-c...,"'AI PC概念股', 'Apple概念股', 'CES概念股', 'ESG概念股', 'i..."
4,2520,冠德,8.96,https://www.wantgoo.com/stock/2520/technical-c...,"'CSR(企業社會責任)概念股', '桃園航空城概念股', '都市更新概念股', '傳產內需股'"
5,2608,嘉里大榮,15.02,https://www.wantgoo.com/stock/2608/technical-c...,"'大陸收成股', '光棍節(雙11)受惠概念股', '物流概念股', '桃園航空城概念股',..."
6,2610,華航,8.50,https://www.wantgoo.com/stock/2610/technical-c...,"'ESG概念股', '桃園航空城概念股', '基金重押股', '陸客來台概念股', '暑假概..."
7,2618,長榮航,7.88,https://www.wantgoo.com/stock/2618/technical-c...,"'ESG概念股', '一帶一路概念股', '台幣貶值受惠概念股', '桃園航空城概念股', ..."
8,4167,松瑞藥,11.84,https://www.wantgoo.com/stock/4167/technical-c...,'防疫概念股'
9,4763,材料*-KY,6.18,https://www.wantgoo.com/stock/4763/technical-c...,"'AI智慧眼鏡概念股', '台積電概念股', '臺灣中型100指數成分股', '臺灣發達指數..."


## 輸出結果至Google sheet

In [21]:
# Google SpreadSheet 公用程式初始化
gspread_mgr = gsm.GspreadManager()
gspread_wb = gspread_mgr.get_spreadsheet(GSPERAD_SHEET_KEY)

print(f"更新Google 表單：{gspread_wb.title}，工作表：{OUTPUT_GSHEET_NAME}")

更新Google 表單：台股選股清單，工作表：選股清單01


In [22]:
# 刪除再重建工作表
gspread_mgr.recreate_worksheet(GSPERAD_SHEET_KEY, OUTPUT_GSHEET_NAME)

In [23]:
# 更新工作表資料
gspread_mgr.update_worksheet_values(
    GSPERAD_SHEET_KEY, 
    OUTPUT_GSHEET_NAME, 
    [merged_df.columns.values.tolist()] + merged_df.values.tolist()
)

In [24]:
# Update 報表更新日期資訊
now_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Update the cell with the current datetime
info_sheet = gspread_mgr.get_worksheet(GSPERAD_SHEET_KEY, "更新日誌")
info_sheet.update_cell(1, 2, now_str)

{'spreadsheetId': '1ovvtmFtsc9Se8r6px2NRferACDckIoOnZJ2__AUwyO8',
 'updatedRange': "'更新日誌'!B1",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [25]:
# 計算總執行時間
end_time = datetime.now()
total_time = end_time - start_time

print(f"分析結束時間: {end_time}")
print(f"總執行時間: {total_time}")

分析結束時間: 2026-02-27 16:24:32.504348
總執行時間: 0:00:10.540167
